In [1]:
from Preprocessing.MECO_data_split import MECODataSplit
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:
dataset = MECODataSplit(['ru'], target_rows=['Target_Ave'])
X, y = dataset.split_by_unique_values(split_cols=['SubjectID', 'Text_ID'],
                                      include_cols='fix',
                                      test_size=0, fix_threshold=0,
                                      resample='truncate', series_length=200)

In [3]:
print(X.shape, y.shape)

(331, 200, 3) (331, 1)


In [9]:
y

array([[4.12950475],
       [4.12950475],
       [4.12950475],
       [4.12950475],
       [4.07598004],
       [4.07598004],
       [3.92895022],
       [3.92895022],
       [3.92895022],
       [3.92895022],
       [3.92895022],
       [3.92895022],
       [3.92895022],
       [3.92895022],
       [3.78304404],
       [3.78304404],
       [3.78304404],
       [3.78304404],
       [2.75138426],
       [2.75138426],
       [2.75138426],
       [2.75138426],
       [2.75138426],
       [2.75138426],
       [4.12139712],
       [3.9045029 ],
       [3.9045029 ],
       [3.9045029 ],
       [3.9045029 ],
       [3.9045029 ],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [3.64499482],
       [4.29654188],
       [4.29654188],
       [2.93138228],
       [2.93138228],
       [2.93138228],
       [2.93138228],
       [2.93138228],
       [2.931

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [11]:
y = y['0']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [5]:
def train_model(conv_filters=64, kernel_size=3, strides=1, dropout_rate=0.5, pool_size=2, epochs=10, batch_size=32):
    model = Sequential()
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, strides=strides,
                     activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, strides=strides, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

    return model

In [6]:
cnn = train_model(epochs=15, batch_size=32, dropout_rate=0.5, conv_filters=64, kernel_size=3)

Epoch 1/15


2023-04-06 17:14:30.698910: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


9/9 [==============================] - 0s 10ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 2/15
9/9 [==============================] - 0s 11ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 3/15
9/9 [==============================] - 0s 10ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 4/15
9/9 [==============================] - 0s 9ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 5/15
9/9 [==============================] - 0s 9ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 6/15
9/9 [==============================] - 0s 9ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 7/15
9/9 [==============================] - 0s 10ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 8/15
9/9 [==============================] - 0s 12ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 9/15
9/9 [==============================] - 0s 10ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epoch 10/15
9/9 [==============================] - 0s 10ms/step - loss: 6.2278 - accuracy: 0.0000e+00
Epo

In [ ]:
loss, accuracy = cnn.evaluate(X_train, y_train)